In [56]:
# read in data from excel file to dataframe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import re
import datetime
import time
import math
from scipy import stats
from scipy.stats import norm
from scipy.stats import t

#===================== Independent Constants =====================
P = 101.20 # (k Pa) Atmospheric pressure
zm = 50.00 # (m) Measurement height for wind speed
k = 0.4 # (none) von Karman's constant
esurface = 0.95 # (none) Emissivity of crop
sigma = 5.67E-08 # (W m-2 K-4) Stefan-Boltzmann constant
KR = 200 # (W m-2) Parameter in Equ (24.2)
KD1 = -0.307 # (kPa-1) Parameter in Equ (24.6) 
KD2 = 0.019 # (kPa-2) Parameter in Equ (24.6) 
TL = 273.00 # (K) Parameter in Equ (24.4) and Equ (24.5) 
T0 = 293.00 # (K) Parameter in Equ (24.4) and Equ (24.5) 
TH = 313.00 # (K) Parameter in Equ (24.4) and Equ (24.5) 
KM1 = 3.36E-04 # (none) Parameter in Equ (24.6) 
KM2 = -0.10 # (mm-1) Parameter in Equ (24.6) 
ra = 1.23 # (kg m-3) Moist Air density
cp = 1013.00 # (J kg-1 K-1) Specific heat capacity of air at constant pressure
gc = 1.00 # (none) Canopy cover factor 
aT = 1.00 # (none) Parameter in temperature stress factor 
zo_prime = 0.003 # (m) Aerodynamic roughness of soil 

#===================== Crop Dependent Constants =====================

forest_constants = {
    "LAI": {"value": 4.00, "unit": "none", "description": "Leaf Area Index"},
    "h": {"value": 20.00, "unit": "m", "description": "Canopy Height"},
    "a": {"value": 0.12, "unit": "none", "description": "Albedo"},
    "g0": {"value": 15.00, "unit": "mm s-1", "description": "Canopy Specific Constant"},
    "SMo": {"value": 80.00, "unit": "mm", "description": "Maximum soil moisture accessible to roots. Parameter in Equ (24.6)"},
    "SMinit": {"value": 40.00, "unit": "mm", "description": "Initial Root-accessible soil moisture"},
    "S": {"value": 4.00, "unit": "mm", "description": "?"},
    "d": {"value": 14.644, "unit": "m", "description": "Zero plane displacement height"},
    "z0": {"value": 1.607, "unit": "m", "description": "Aerodynamic roughness of crop"}
}

grass_constants = {
    "LAI": {"value": 2.00, "unit": "none", "description": "Leaf Area Index"},
    "h": {"value": 0.12, "unit": "m", "description": "Canopy Height"},
    "a": {"value": 0.23, "unit": "none", "description": "Albedo"},
    "g0": {"value": 30.00, "unit": "mm s-1", "description": "Canopy Specific Constant"},
    "SMo": {"value": 40.00, "unit": "mm", "description": "Parameter in Equ (24.6)"},
    "SMinit": {"value": 10.00, "unit": "mm", "description": "Initial Root-accessible soil moisture"},
    "S": {"value": 2.00, "unit": "mm", "description": "?"},
    "d": {"value": 0.077, "unit": "m", "description": "Zero plane displacement height"},
    "z0": {"value": 0.013, "unit": "m", "description": "Aerodynamic roughness of crop"}
}

# Convert dictionaries to DataFrames
forest_constants = pd.DataFrame.from_dict(forest_constants, orient='index')
grass_constants= pd.DataFrame.from_dict(grass_constants, orient='index')


#==================== Model forcing data ==========================
# Read in data from excel file
excel_file = "Lab1.xlsx"
# Read the data, skipping rows so that headers start from row 11 (indexing starts from 0)
# header=0 in this context means the header is taken from the first row read (after skipping)
forest_data = pd.read_excel(excel_file, header=0, sheet_name = "Forest", skiprows=10)
#read grass data form sheet labeled Grass
grass_data = pd.read_excel(excel_file, sheet_name="Grass", header=0, skiprows=10) 

#print headers
print(grass_data.columns)


Index(['Total Hour', ' Hour', 'S', 'LW_D', 'Ta', 'um', 'q', 'P', 'ra', 'e',
       'esat(T)', 'D', 'T', 'gR', 'gD', 'gT', 'gSM', 'SMlast', 'gs', 'rs',
       'Lu', 'Rn', 'lambda', 'delta', 'psyc', 'Cinterim', 'Cactual', 'Dcanopy',
       'C/S', 'lambda*EI (C=Cactual)', 'Cfinal', 'lambda*ET', 'lambda*Etotal',
       'H', 'Tsurface', 'SMnew'],
      dtype='object')


c:\Users\adunw\miniconda3\envs\EcoHydro\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
c:\Users\adunw\miniconda3\envs\EcoHydro\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [57]:
# Part A: Aerodynamic Parameterization. 

def calc_d(h, LAI):
    """
    Calculate zero plane displacement height
    params:
    h: canopy height (m)
    LAI: leaf area index
    """
    return 1.1*h*np.log(1 + (LAI/5)**0.25) # TH Eq 22.2 

def calc_zo(h, d, LAI):
    """
    Calculate aerodynamic roughness of crop
    params: 
    h: canopy height (m)
    d: zero plane displacement height (m)
    LAI: leaf area index
    """
    if LAI > 1:
        return 0.3 * h * (1 - d / h) # TH Eq 22.4
    else:
        return zo_prime + 0.29 * h * (LAI / 5) ** 0.5 # TH Eq 22.3
    
def calc_ra(zm, d, zo, um):
    """
    Calculate aerodynamic resistance
    params:
    zm: measurement height for wind speed (m)
    d: zero plane displacement height (m)
    zo: aerodynamic roughness of crop  (m)
    um: wind speed (m/s) (given in forcing data)
    """
    return 1 / (k**2 * um) * np.log((zm - d) / zo) * np.log((zm - d) / zo) # TH Eq 22.1

# populate the dataframes with calculated values
forest_constants['d'] = calc_d(forest_constants.loc['h']['value'], forest_constants.loc['LAI']['value'])
forest_constants['z0'] = calc_zo(forest_constants.loc['h']['value'], forest_constants.loc['d']['value'], forest_constants.loc['LAI']['value'])
grass_constants['d'] = calc_d(grass_constants.loc['h']['value'], grass_constants.loc['LAI']['value'])
grass_constants['z0'] = calc_zo(grass_constants.loc['h']['value'], grass_constants.loc['d']['value'], grass_constants.loc['LAI']['value'])

forest_data['ra'] = calc_ra(zm, forest_constants.loc['d']['value'], forest_constants.loc['z0']['value'], forest_data['um'])
grass_data['ra'] = calc_ra(zm, grass_constants.loc['d']['value'], grass_constants.loc['z0']['value'], grass_data['um'])



In [62]:
import numpy as np
forest_constants = {
    "LAI": {"value": 4.00, "unit": "none", "description": "Leaf Area Index"},
    "h": {"value": 20.00, "unit": "m", "description": "Canopy Height"},
    "a": {"value": 0.12, "unit": "none", "description": "Albedo"},
    "g0": {"value": 15.00, "unit": "mm s-1", "description": "Canopy Specific Constant"},
    "SMo": {"value": 80.00, "unit": "mm", "description": "Maximum soil moisture accessible to roots. Parameter in Equ (24.6)"},
    "SMinit": {"value": 40.00, "unit": "mm", "description": "Initial Root-accessible soil moisture"},
    "S": {"value": 4.00, "unit": "mm", "description": "?"},
    "d": {"value": 14.644, "unit": "m", "description": "Zero plane displacement height"},
    "z0": {"value": 1.607, "unit": "m", "description": "Aerodynamic roughness of crop"}
}

grass_constants = {
    "LAI": {"value": 2.00, "unit": "none", "description": "Leaf Area Index"},
    "h": {"value": 0.12, "unit": "m", "description": "Canopy Height"},
    "a": {"value": 0.23, "unit": "none", "description": "Albedo"},
    "g0": {"value": 30.00, "unit": "mm s-1", "description": "Canopy Specific Constant"},
    "SMo": {"value": 40.00, "unit": "mm", "description": "Parameter in Equ (24.6)"},
    "SMinit": {"value": 10.00, "unit": "mm", "description": "Initial Root-accessible soil moisture"},
    "S": {"value": 2.00, "unit": "mm", "description": "?"},
    "d": {"value": 0.077, "unit": "m", "description": "Zero plane displacement height"},
    "z0": {"value": 0.013, "unit": "m", "description": "Aerodynamic roughness of crop"}
}

for constants in [forest_constants, grass_constants]:
    h_value = constants['h']['value']
    LAI_value = constants['LAI']['value']
    
    # Calculate 'd' and 'z0' using the function and update the dictionary
    constants['d']['value'] = calc_d(h_value, LAI_value)
    constants['z0']['value'] = calc_zo(h_value, constants['d']['value'], LAI_value)

from openpyxl import load_workbook

excel_file = "Lab1.xlsx"

# Load the workbook and the specific sheet
wb = load_workbook(filename=excel_file, data_only=True)

# Function to read data from a sheet into a dictionary
def read_sheet_to_dict(wb, sheet_name, start_row=11):
    sheet = wb[sheet_name]
    data = {}
    # Assuming the first row of data (after skipped rows) contains headers
    headers = [cell.value for cell in sheet[start_row]]
    for row in sheet.iter_rows(min_row=start_row+1, values_only=True):
        for header, value in zip(headers, row):
            if header not in data:
                data[header] = []
            data[header].append(value)
    return data

# Read data from the "Forest" and "Grass" sheets into dictionaries
forest_data = read_sheet_to_dict(wb, "Forest", start_row=11)
grass_data = read_sheet_to_dict(wb, "Grass", start_row=11)

# To print headers (keys of the dictionary)
print("Forest Data Columns:", list(forest_data.keys()))
print("Grass Data Columns:", list(grass_data.keys()))


c:\Users\adunw\miniconda3\envs\EcoHydro\Lib\site-packages\openpyxl\reader\excel.py:237: UserWarning: Unknown extension is not supported and will be removed
  ws_parser.bind_all()


Forest Data Columns: ['Total Hour', ' Hour', 'S', 'LW_D', 'Ta', 'um', 'q', 'P', 'ra', 'e', 'esat(T)', 'D', 'T', 'gR', 'gD', 'gT', 'gSM', 'SMlast', 'gs', 'rs', 'Lu', 'Rn', 'lambda', 'delta', 'psyc', 'Cinterim', 'Cactual', 'Dcanopy', 'C/S', 'lambda*EI (C=Cactual)', 'Cfinal', 'lambda*ET', 'lambda*Etotal', 'H', 'Tsurface', 'SMnew']
Grass Data Columns: ['Total Hour', ' Hour', 'S', 'LW_D', 'Ta', 'um', 'q', 'P', 'ra', 'e', 'esat(T)', 'D', 'T', 'gR', 'gD', 'gT', 'gSM', 'SMlast', 'gs', 'rs', 'Lu', 'Rn', 'lambda', 'delta', 'psyc', 'Cinterim', 'Cactual', 'Dcanopy', 'C/S', 'lambda*EI (C=Cactual)', 'Cfinal', 'lambda*ET', 'lambda*Etotal', 'H', 'Tsurface', 'SMnew', None]


In [58]:
# Part B: Surface Resistance Parameterization

def calc_gR(S, KR):
    """
    Calculate resistance to soil heat flux
    params:
    S: Incoming shortwave radiation (W m-2)
    KR: Parameter in Equ (24.2)
    """
    gR =  S * (1000 + KR) / (1000 * (S + KR)) # Eq. 24.2 TH
    return gR

def calc_gD(D, KD1, KD2):
    """
    Calculate resistance to vapor pressure deficit
    params:
    D: Vapor pressure deficit (kPa)
    KD1: Parameter in Equ (24.6) 
    KD2: Parameter in Equ (24.6) 
    """
    gD = 1 + KD1 * D + KD2 * D**2 # Eq. 24.3 TH
    return gD
    
def calc_gT(T):
    """
    Calculate resistance to temperature
    params:
    T: Temperature (K)
    """
    gT = (T - TL) * (TH - T)**aT / ((T - TL) * (TH - T0)**aT) # Eq. 24.4 TH
    return gT

def calc_gSM(SM, SMo, KM1, KM2): 
    """
    Calculate resistance to soil moisture flux
    params:
    SM: Soil moisture (mm)
    SMo: Maximum soil moisture accessible to roots. Parameter in Equ (24.6)
    KM1: Parameter in Equ (24.6)
    KM2: Parameter in Equ (24.6)
    """
    gSM = 1 - KM1 * np.exp(KM2 * (SM - SMo))
    return gSM

def calc_rs(gR, gD, gT, gSM):
    """
    Calculate total surface resistance
    params:
    gR: resistance to soil heat flux
    gD: resistance to vapor pressure deficit
    gT: resistance to temperature
    gSM: resistance to soil moisture flux
    """
    g0 = forest_constants.loc['g0']['value']
    return 1 / (gR * gD * gT * gSM * g0 * gc) # Eq. 24.1 TH
    
def calc_esat(T):
    """
    Calculate saturation vapor pressure from temperature
    T: temperature in degrees Celsius
    return: saturation vapor pressure in kPa
    """
    esat = 0.6108 * np.exp(17.27 * T / (T + 237.3)) # Eq. 2.17 TH
    return esat

def calc_e(q, P):
    """
    Calculate vapor pressure from specific humidity and pressure
    q: specific humidity in kg/kg
    P: pressure in kPa
    return: vapor pressure in kPa
    """
    e = P * q / 0.622 # Eq 2.9 TH
    return e

def calc_delta(T):
    """
    Calculate gradient of the saturation vapor pressure curve from saturation vapor pressure and temperature
    e_sat: saturation vapor pressure in kPa
    T: temperature in degrees Celsius
    return: slope of the saturation vapor pressure curve in kPa/°C
    """
    e_sat = calc_esat(T)
    delta = 4098 * e_sat / (T + 237.3)**2 # Eq 2.18 TH
    return delta


# populate the dataframes with calculated values
forest_data['gR'] = calc_gR(forest_data['S'], KR)
forest_data['gD'] = calc_gD(forest_data['D'], KD1, KD2)
forest_data['gT'] = calc_gT(forest_data['T'])
forest_data['gSM'] = calc_gSM(forest_data['SM'], forest_constants.loc['SMo']['value'], KM1, KM2)
#check if gR, gD, gT, or gSM are less than 0 and if so set to 0
forest_data.loc[forest_data['gR'] < 0, 'gR'] = 0
forest_data.loc[forest_data['gD'] < 0, 'gD'] = 0
forest_data.loc[forest_data['gT'] < 0, 'gT'] = 0
forest_data.loc[forest_data['gSM'] < 0, 'gSM'] = 0

forest_data['rs'] = calc_rs(forest_data['gR'], forest_data['gD'], forest_data['gT'], forest_data['gSM'])
forest_data['e_sat'] = calc_esat(forest_data['T'])
forest_data['e'] = calc_e(forest_data['q'], P)
forest_data['delta'] = calc_delta(forest_data['T'])





KeyError: 'SM'